In [1]:
import pandas as pd

In [30]:
data = pd.read_csv('bigdata.csv')

In [32]:
data['price'].unique()

array(['$$', '0', '$', '$$$', '$$$$', '€€'], dtype=object)

In [4]:
data.groupby(['City', 'BusinessType']).count()

Unnamed: 0  State  name  cat  price  rating  review
City      BusinessType                                                       
austin    restaurants            400    400   400  400    400     400     400
          spas                   400    400   400  400    400     400     400
baltimore privateschools           1      1     1    1      1       1       1
          restaurants            400    400   400  400    400     400     400
          spas                   300    300   300  300    300     300     300
...                              ...    ...   ...  ...    ...     ...     ...
ventura   spas                   180    180   180  180    180     180     180
wichita   restaurants            320    320   320  320    320     320     320
          spas                    80     80    80   80     80      80      80
worcester restaurants            320    320   320  320    320     320     320
          spas                   120    120   120  120    120     120     120

[99 rows x 7 columns]

In [49]:
data['price']=data['price'].apply(lambda x: 2 if (x == "€€") else x)
data['price']=data['price'].apply(lambda x: 1 if (x == "$") else x)
data['price']=data['price'].apply(lambda x: 2 if (x == "$$") else x)
data['price']=data['price'].apply(lambda x: 3 if (x == "$$$") else x)
data['price']=data['price'].apply(lambda x: 4 if (x == "$$$$") else x)

In [56]:
data['price'] = data['price'].astype('int')

In [12]:
data1 = data

In [22]:
data1.drop(data2, inplace=True)

In [61]:
data1['price']=data1['price'].apply(lambda x: 2 if (x == "€€") else x)
data1['price']=data1['price'].apply(lambda x: 1 if (x == "$") else x)
data1['price']=data1['price'].apply(lambda x: 2 if (x == "$$") else x)
data1['price']=data1['price'].apply(lambda x: 3 if (x == "$$$") else x)
data1['price']=data1['price'].apply(lambda x: 4 if (x == "$$$$") else x)
data1['price'] = data1['price'].astype('int')

In [19]:
data2 = list(data.loc[data['price'] == '0'].index)

In [64]:
data1_means = data1.groupby(['City', 'BusinessType']).mean()
data_means = data.groupby(['City', 'BusinessType']).mean()

In [33]:
i = 0
for row in data['price']:
    if row['price'] == '0':
            data['price'][i] = 

SyntaxError: invalid syntax (<ipython-input-33-c9893a5b4dd2>, line 2)

In [73]:
data1_means.reset_index(inplace=True)

In [75]:
data1_means

,City,BusinessType,Unnamed: 0,price,rating,review
0,austin,restaurants,194.677143,1.742857,4.038571,418.960000
1,austin,spas,193.233333,2.066667,4.409524,65.200000
2,baltimore,restaurants,191.135048,1.720257,3.932476,222.086817
3,baltimore,spas,160.316514,2.059633,3.616972,26.798165
4,boise,restaurants,188.362229,1.526316,3.501548,89.266254
...,...,...,...,...,...,...
92,ventura,spas,90.297521,1.933884,4.223140,47.446281
93,wichita,restaurants,161.944030,1.406716,3.404851,44.395522
94,wichita,spas,40.384615,2.096154,3.759615,10.923077
95,worcester,restaurants,157.269962,1.547529,3.433460,83.714829


In [76]:
data_group = data.groupby(['City', 'BusinessType']).count()

In [78]:
data_group.reset_index(inplace=True)

In [81]:
data_group.drop(['State','name','cat','price','rating','review'], axis=1, inplace=True)

In [84]:
data_group = data_group.rename(columns={'Unnamed: 0': 'Count'})

ValueError: Can only compare identically-labeled Series objects

In [91]:
data_group.loc[data_group['BusinessType'] == 'privateschools']


,City,BusinessType,Count


In [90]:
data_group.drop([2,70], inplace=True)

In [96]:
data_group.reset_index(inplace=True)

In [99]:
(data_group['City'] == data1_means['City'])


0     True
1     True
2     True
3     True
4     True
      ... 
92    True
93    True
94    True
95    True
96    True
Name: City, Length: 97, dtype: bool

In [101]:
data1_means = data1_means.drop(['Unnamed: 0', 'review'], axis=1)

In [110]:
df_merged = data1_means.merge(data_group, how='inner', left_index=True, right_index=True)


In [111]:
df_merged

,City_x,BusinessType_x,price,rating,index,City_y,BusinessType_y,Count
0,austin,restaurants,1.742857,4.038571,0,austin,restaurants,400
1,austin,spas,2.066667,4.409524,1,austin,spas,400
2,baltimore,restaurants,1.720257,3.932476,3,baltimore,restaurants,400
3,baltimore,spas,2.059633,3.616972,4,baltimore,spas,300
4,boise,restaurants,1.526316,3.501548,5,boise,restaurants,380
...,...,...,...,...,...,...,...,...
92,ventura,spas,1.933884,4.223140,94,ventura,spas,180
93,wichita,restaurants,1.406716,3.404851,95,wichita,restaurants,320
94,wichita,spas,2.096154,3.759615,96,wichita,spas,80
95,worcester,restaurants,1.547529,3.433460,97,worcester,restaurants,320


In [113]:
new_data = df_merged.drop(['index', 'City_y', 'BusinessType_y'], axis=1)

In [119]:
new_data = new_data.rename(columns={'City_x':'City', 'BusinessType_x':'Type'})

In [121]:
data_spas = new_data.loc[new_data['Type'] == 'spas']

In [122]:
data_rest = new_data.loc[new_data['Type'] == 'restaurants']

In [127]:
data_spas.reset_index(inplace=True)

In [128]:
data_rest.reset_index(inplace=True)

In [124]:
final_db = pd.read_csv("merged_data.csv")

In [130]:
final_db = final_db.drop(['Unnamed: 0', '2016 land area','2016 land area.1','Unnamed: 0.1'], axis=1)

In [132]:
final_db = final_db.rename(columns={'DataValue':'PCI'})

In [136]:
final_db = final_db.sort_values('City')

In [139]:
final_db.reset_index(drop=True, inplace=True)

In [143]:
final_db.drop('index', axis=1, inplace=True)

In [151]:
final_db['City'] = final_db['City'].apply(lambda x: x.lower())

In [153]:
final_db['City'] == data_rest['City']

ValueError: Can only compare identically-labeled Series objects

In [145]:
data_rest

,index,City,Type,price,rating,Count
0,0,austin,restaurants,1.742857,4.038571,400
1,2,baltimore,restaurants,1.720257,3.932476,400
2,4,boise,restaurants,1.526316,3.501548,380
3,6,charleston,restaurants,1.704050,3.864486,400
4,8,clinton,restaurants,1.250000,3.500000,5
5,10,dallas,restaurants,1.853933,3.995787,400
6,12,denton,restaurants,1.447368,3.311404,140
7,14,denver,restaurants,1.838710,4.064516,400
8,16,des moines,restaurants,1.513932,3.537152,400
9,18,durham,restaurants,1.479554,3.466543,300


In [157]:
final_db.drop(25, axis=0, inplace=True)

In [159]:
final_db.reset_index(inplace=True, drop=True)

In [163]:
final_rest = final_db.merge(data_rest, how='inner', left_index=True, right_index=True)

In [166]:
final_rest = final_rest.drop(['index','City_y', 'Type'], axis=1)

In [168]:
final_rest['Count'] = final_rest['Count']/final_rest['2018estimate']

In [170]:
final_rest = final_rest.drop(['2010Census'], axis=1)

In [172]:
final_rest = final_rest.rename(columns={'City_x':'City'})

In [173]:
final_rest.to_csv('final_rest.csv')

In [175]:
final_spa = final_db.merge(data_spas, how='inner', left_index=True, right_index=True)

In [177]:

final_spa = final_spa.drop(['index','City_y', 'Type'], axis=1)
final_spa['Count'] = final_spa['Count']/final_spa['2018estimate']
final_spa = final_spa.drop(['2010Census'], axis=1)
final_spa = final_spa.rename(columns={'City_x':'City'})

In [179]:
final_spa.to_csv('final_spa.csv', index=False)